In [1]:
# Install lxml, which allows me to read the html file with the Canada data
!pip install lxml 

# numpy and pandas allow me to work with the data
import numpy as np 
import pandas as pd

# The following code below allows me to use SQL with python
# I find using SQL to merge and join dataframes easier, and I'll explain the SQL code further down
!pip install pandasql 
from pandasql import sqldf

# Import folium to plot the points for my map
import folium 

# Libraries Installed in the Above Cell

Now, it's time to download the data from the webpage.

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dataset_list = pd.read_html(url, header=0)

Let's view the data!

In [3]:
# I'm calling my dataframe 'postal_codes_df' and creating a list from the data pulled from the webpage
postal_codes_df = dataset_list[0]

In [4]:
#the head() function allows me to see a certain number of rows of my dataframe
postal_codes_df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


We can see that the data has several rows with 'Not Assigned' - let's clean up the data!

# Cleaning the Data
Only process rows with an assigned borough.

This means we should filter out all rows where Borough is "Not assigned" using pandas.

In [7]:
postal_codes_df = postal_codes_df.query("Borough != 'Not assigned'")


Now let's view the partially cleaned data. 

In [8]:
postal_codes_df.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


We still need to replace the 'Not assigned' in the Neighbourhood column with the Borough.

In [9]:
postal_codes_df = postal_codes_df.groupby(['Postcode', 'Borough'], as_index=False, sort=False)
postal_codes_df = postal_codes_df.agg(','.join)
postal_codes_df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Now we make the Neighbourhoods that are 'Not assigned' equal to the Borough value. Let's see which rows meet this exception:

In [10]:
postal_codes_df.query("Neighbourhood == 'Not assigned'")

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


There's only one row that meets this exception, so I will replace that 'Not assigned' with the Borough name.

In [11]:
postal_codes_df.loc[postal_codes_df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = "Queen's Park"
postal_codes_df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Now let's create a new dataframe with longitude and latitude!

In [12]:
geocode_df=pd.read_csv('Geospatial_Coordinates.csv')
geocode_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's merge the postal_codes_df with our new geospatial dataframe!

In order to join the two dataframes, I am using pandas sql (note in the first cell I imported pandasql, which allows me to use SQL within python). I found this to be an easier and cleaner way to join the two dataframes successfully. We weren't taught to do this, but I've learned SQL and find it more straighforward for joining. 

In [13]:
query = """
    SELECT p.*, Latitude, Longitude
    FROM postal_codes_df as p
    INNER JOIN geocode_df as g ON p.Postcode=g.'Postal Code'
"""

postal_codes_df=sqldf(query)
postal_codes_df.head(12)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


What I basically told the code to do is select everything from the postal_codes_df and only Latitude and Longitude from the geocode_df. An inner join in SQL merges the matching values in each table (so since both post codes are the same in both dataframes, they stay the same and become one column, 'Postcode').

Now I want to plot the latitude and longitude points on a map visualisation!

In [14]:
# the location chosen allows us to see all the plotted point from the data with the zoom level 11
# I then create a loop with 'i' in order to plot all the points simultaneously; 
# the len function allows the code to read all the rows from the dataframe
postal_map = folium.Map(location=[43.753259, -79.329656], zoom_start = 11)
for i in range(0, len(postal_codes_df)):
    folium.Marker([postal_codes_df.iloc[i]['Latitude'],postal_codes_df.iloc[i]['Longitude']]).add_to(postal_map)

postal_map

Now I need to segment and cluster the data so we can distinguish neighbourhoods!

I only want boroughs that include the word 'Toronto' in their name. I'm going to use the LIKE function to create a dataframe of all the boroughs with the name Toronto: 

In [15]:
# I'm still using pandasql to run SQL queries in Python, so code is different than python code

query = """
    SELECT 
        *
    FROM postal_codes_df 
    WHERE Borough like '%Toronto%'
"""
Toronto_df = sqldf(query)
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Then I want to create a dataframe with the rest of the boroughs:

In [16]:
query = """
    SELECT 
        *
    FROM postal_codes_df 
    WHERE Borough not like '%Toronto%'
"""
not_Toronto_df = sqldf(query)
not_Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


Next, I want to create the folium map. The code below will first loop through the "Toronto" set coordinates and plot them red on the map. Then it will loop through the non-Toronto set of coordinates and plot them blue on the map. Then it shows the map.


In [17]:
postal_map = folium.Map(location=[43.753259, -79.329656], zoom_start = 11)

for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.3,
        parse_html=False).add_to(postal_map)     
    
for lat, lng, label in zip(not_Toronto_df['Latitude'], not_Toronto_df['Longitude'], not_Toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
        parse_html=False).add_to(postal_map)     

postal_map
# You should be able to click on different dots and it will show you those clusters of boroughs!


If you click on the dots, you can see the boroughs that have been clustered. I hope you enjoyed segmenting and clustering neighbourhoods and boroughs with me!

In [18]:
postal_codes_df.shape

(103, 5)